# --- Day 7: The Treachery of Whales ---

## Part 1

A giant whale has decided your submarine is its next meal, and it's much faster than you are. There's nowhere to run!

Suddenly, a swarm of crabs (each in its own tiny submarine - it's too deep for them otherwise) zooms in to rescue you! They seem to be preparing to blast a hole in the ocean floor; sensors indicate a **massive underground cave system** just beyond where they're aiming!

The crab submarines all need to be aligned before they'll have enough power to blast a large enough hole for your submarine to get through. However, it doesn't look like they'll be aligned before the whale catches you! Maybe you can help?

There's one major catch - crab submarines can only move horizontally.

You quickly make a list of the **horizontal position of each crab** (your puzzle input). Crab submarines have limited fuel, so you need to find a way to make all of their horizontal positions match while requiring them to spend as little fuel as possible.

For example, consider the following horizontal positions:

```
16,1,2,0,4,2,7,1,2,14
```

This means there's a crab with horizontal position 16, a crab with horizontal position 1, and so on.

Each change of 1 step in horizontal position of a single crab costs 1 fuel. You could choose any horizontal position to align them all on, but the one that costs the least fuel is horizontal position 2:

```
    Move from 16 to 2: 14 fuel
    Move from 1 to 2: 1 fuel
    Move from 2 to 2: 0 fuel
    Move from 0 to 2: 2 fuel
    Move from 4 to 2: 2 fuel
    Move from 2 to 2: 0 fuel
    Move from 7 to 2: 5 fuel
    Move from 1 to 2: 1 fuel
    Move from 2 to 2: 0 fuel
    Move from 14 to 2: 12 fuel
```

This costs a total of 37 fuel. This is the cheapest possible outcome; more expensive outcomes include aligning at position 1 (41 fuel), position 3 (39 fuel), or position 10 (71 fuel).

Determine the horizontal position that the crabs can align to using the least fuel possible. **How much fuel must they spend to align to that position**?



In [1]:
import numpy as np
import altair as alt
import pandas as pd

In [2]:
def find_min_fuel(crab_locations):
    
    crab_locations = np.array(crab_locations)
    max_val = max(crab_locations)
    min_val = min(crab_locations)
    
    fuel_cost_dict = dict()

    for point in range(min_val, max_val+1):
        fuel_cost_dict[point] = sum( abs(crab_locations - point) )
    
    min_key = min(fuel_cost_dict, key=fuel_cost_dict.get)
    return min_key, fuel_cost_dict[min_key]

### Run function with the test input


In [3]:
test_input = [16,1,2,0,4,2,7,1,2,14]

known_answer = 37

min_key, answer = find_min_fuel(test_input)

if known_answer == answer:
    print("Looks like its working!!")
else:
    print("There is something wrong... :(")

Looks like its working!!


### Visualize the crabs and the minimum fuel location...

In [4]:
frame = pd.DataFrame({
    "points" : test_input,
    "idx" : range(len(test_input)),
    "min_loc" : [known_answer] * len(test_input)
})

crabs = alt.Chart(frame).mark_point().encode(
    x = alt.X('idx:O'),
    y = alt.Y('points:Q')
).properties(height = 300, width = 600)

line = alt.Chart(pd.DataFrame({'y': [min_key]})).mark_rule().encode(y='y')


alt.layer(crabs, line, title = "Crabs and minimum fuel location")

alt.LayerChart(...)

### Run it again on the puzzle input...

In [5]:
data_file = "../data/07_data.txt"

with open(data_file, "r") as f:
    puzzle_input = [line.split(",") for line in f][0]

puzzle_input = list(map(int, puzzle_input))

min_key, answer = find_min_fuel(puzzle_input)
answer

340987

In [6]:
min_key

343

## Part 2

The crabs don't seem interested in your proposed solution. Perhaps you misunderstand crab engineering?

As it turns out, crab submarine engines don't burn fuel at a constant rate. **Instead, each change of 1 step in horizontal position costs 1 more unit of fuel than the last: the first step costs 1, the second step costs 2, the third step costs 3, and so on.**

As each crab moves, moving further becomes more expensive. This changes the best horizontal position to align them all on; in the example above, this becomes 5:

```
    Move from 16 to 5: 66 fuel
    Move from 1 to 5: 10 fuel
    Move from 2 to 5: 6 fuel
    Move from 0 to 5: 15 fuel
    Move from 4 to 5: 1 fuel
    Move from 2 to 5: 6 fuel
    Move from 7 to 5: 3 fuel
    Move from 1 to 5: 10 fuel
    Move from 2 to 5: 6 fuel
    Move from 14 to 5: 45 fuel
```

This costs a total of **168** fuel. This is the new cheapest possible outcome; the old alignment position (2) now costs 206 fuel instead.

**Determine the horizontal position that the crabs can align to using the least fuel possible so they can make you an escape route! How much fuel must they spend to align to that position?**


### New cost function...

In [7]:
difference_cache = dict()

def repetitive_addition(n):
    """
    This is a recursive cost function which must
    run on the difference for each 
    """
    if n == 0:
        return 0

    if n == 1:
        return 1
    else:
        return n + repetitive_addition(n-1)

def find_min_fuel_factorial(crab_locations):
    """
    The main change to this function is:
        1. The cost function (above) is much more expensive
        2. We use a cache to record previously calculated fuel
            cost values, making this MUCH more efficient.

    The idea here is that the space of possible distances crabs
    can travel is well defined. Between...
        min fuel cost = 0 and
        max fuel cost = max_crab_location - min_crab_location
    ... so there is a very high chance that lots of crabs will
    need to travel the same distances. As we calculate the cost
    of traveling n moves (with the above recursive function), we
    can then simply save that value in the `difference_cache` so
    we don't need to calculate it again the next time we need it.
    """
    
    # Initialize a cost difference cache.
    difference_cache = dict()
    
    # Initialize final fuel cost dictionary
    fuel_cost_dict = dict()
    
    # Convert input to array and get search bounds
    crab_locations = np.array(crab_locations)
    max_val = max(crab_locations)
    min_val = min(crab_locations)
    
    # Search through  
    for point in range(min_val, max_val+1):
        abs_val_diff = abs(crab_locations - point)
        
        fuel_cost = 0
        for val in abs_val_diff:
            
            # Try to draw on our fuel cost difference cache
            if val in difference_cache:
                fuel_cost += difference_cache[val]
            
            # Otherwise, we update it with the newly calculated cost
            else:
                indv_fuel_cost = repetitive_addition(val)
                difference_cache[val] = indv_fuel_cost
                fuel_cost += indv_fuel_cost

        fuel_cost_dict[point] = fuel_cost

    min_key = min(fuel_cost_dict, key=fuel_cost_dict.get)
    return min_key, fuel_cost_dict[min_key]

### Run on test input

In [8]:
known_answer = 168

min_key, answer = find_min_fuel_factorial(test_input)

if known_answer == answer:
    print("Looks like its working!!")
else:
    print("There is something wrong... :(")

Looks like its working!!


### Run on puzzle input

In [9]:
min_key

5

In [10]:
min_key, answer = find_min_fuel_factorial(puzzle_input)

print(answer)

96987874


In [11]:
frame = pd.DataFrame({
    "points" : puzzle_input,
    "idx" : range(len(puzzle_input)),
    "min_loc" : [answer] * len(puzzle_input)
})

crabs = alt.Chart(frame).transform_calculate(
    # Generate Gaussian jitter with a Box-Muller transform
    jitter='sqrt(-2*log(random()))*cos(2*PI*random())'
).mark_point().encode(
    x = alt.X(
        'jitter:Q'
    ),
    y = alt.Y(
        'points:Q',
        axis = alt.Axis(
            title = ["Crab","Location"],
            titleAngle = 0,
            titlePadding = 20
        )
    )
).properties(height = 300, width = 800)

line = alt.Chart(pd.DataFrame({'y': [min_key]})).mark_rule().encode(y='y')
text = alt.Chart(pd.DataFrame({
    'y': [min_key+30],
    'x': [3],
    "text" : ["Minimum Fuel Location"]
})).mark_text().encode(
    y='y',
    x ="x",
    text = "text"
)


alt.layer(crabs, line, text, title = ["Crabs and minimum fuel location","(marked with the horizontal line)"])

alt.LayerChart(...)